# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
S. Li  ->  S. Li  |  ['S. Li']
M. Hobson  ->  M. Hobson  |  ['M. Hobson']
Arxiv has 60 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2310.08023


extracting tarball to tmp_2310.08023...

 done.


A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']


...

extracting tarball to tmp_2310.08180...

 done.


S. Li  ->  S. Li  |  ['S. Li']


list index out of range
Retrieving document from  https://arxiv.org/e-print/2310.08490


extracting tarball to tmp_2310.08490... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.08023-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.08023) | **VERTICO and IllustrisTNG: The spatially resolved effects of environment  on galactic gas**  |
|| A. R. H. Stevens, et al. -- incl., <mark>A. Pillepich</mark> |
|*Appeared on*| *2023-10-13*|
|*Comments*| *9 pages, 4 figures, accepted in ApJL*|
|**Abstract**| It has been shown in previous publications that the TNG100 simulation quantitatively reproduces the observed reduction in each of the total atomic and total molecular hydrogen gas for galaxies within massive halos, i.e.~dense environments. In this Letter, we study how well TNG50 reproduces the resolved effects of a Virgo-like cluster environment on the gas surface densities of satellite galaxies with $m_* > \! 10^9\,{\rm M}_\odot$ and ${\rm SFR} \! > 0.05\,{\rm M}_\odot\,{\rm yr}^{-1}$. We select galaxies in the simulation that are analogous to those in the HERACLES and VERTICO surveys, and mock-observe them to the common specifications of the data. Although TNG50 does not quantitatively match the observed gas surface densities in the centers of galaxies, the simulation does qualitatively reproduce the trends of gas truncation and central density suppression seen in VERTICO in both HI and H$_2$. This result promises that modern cosmological hydrodynamic simulations can be used to reliably model the post-infall histories of cluster satellite galaxies. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.08180-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.08180) | **A High-Mass Young Star-forming Core Escaping from Its Parental Filament**  |
|| Z. Ren, et al. -- incl., <mark>S. Li</mark> |
|*Appeared on*| *2023-10-13*|
|*Comments*| *15 pages, 6 figures, accepted for publication in the Astrophysical Journal*|
|**Abstract**| We studied the unique kinematic properties in massive filament G352.63-1.07 at $10^3$-AU spatial scale with the dense molecular tracers observed with the Atacama Large Millimeter/submillimeter Array (ALMA). We find the central massive core M1 (12 $M_\odot$) being separated from the surrounding filament with a velocity difference of $v- {v}_{sys}=-2$ km/s and a transverse separation within 3 arcsec. Meanwhile, as shown in multiple dense-gas tracers, M1 has a spatial extension closely aligned with the main filament and is connected to the filament towards its both ends. M1 thus represents a very beginning state for a massive young star-forming core escaping from the parental filament, within a time scale of $\sim 4000$ years. Based on its kinetic energy ($3.5\times10^{44}$ erg), the core escape is unlikely solely due to the original filament motion or magnetic field, but requires more energetic events such as a rapid intense anisotropic collapse. The released energy also seems to noticeably increase the environmental turbulence. This may help the filament to become stabilized again. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.08490-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.08490) | **Balancing ACT: weighing prior dependency and global tensions of DR6  lensing with other datasets**  |
|| A. Ormondroyd, W. Handley, <mark>M. Hobson</mark>, A. Lasenby |
|*Appeared on*| *2023-10-13*|
|*Comments*| *8 pages, 2 figures. Nested sampling chains and analysis code available at this https URL Comments welcome!*|
|**Abstract**| We provide a complementary nested sampling analysis for the Atacama Cosmology Telescope lensing data release 6. This allows the quantification of global consistency statistics between ACT lensing and alternative datasets. In the context of flat $\Lambda$CDM, we find no inconsistency between ACT, Baryonic Acoustic Oscillations, Planck anisotropies, weak lensing datasets, or NPIPE lensing. As part of our analysis, we also investigate the effect of the prior widths used in the ACT analysis and find that the headline results are quantitatively but not qualitatively affected by the chosen priors. We use both Bayes factors and the suspiciousness statistic to quantify the possibility of tension, and find suspiciousness unsuitable in the case of strong agreement between ACT DR6 and NPIPE. Nested sampling provides a competitive alternative to Metropolis Hastings and we recommend it be used alongside existing analyses. We release the chains and plotting source for the analysis using anesthetic. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2310.08023.md
    + _build/html/tmp_2310.08023/./Scaling_relations.png
    + _build/html/tmp_2310.08023/./Individual_sequences_HI.png
    + _build/html/tmp_2310.08023/./HIH2frac_TNG100_TNG50.png
exported in  _build/html/2310.08180.md
    + _build/html/tmp_2310.08180/./m1_blue3.png
    + _build/html/tmp_2310.08180/./spec_img.png
    + _build/html/tmp_2310.08180/./pv_3mol.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# VERTICO and IllustrisTNG: The spatially resolved effects of environment on galactic gas

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.08023-b31b1b.svg)](https://arxiv.org/abs/2310.08023)<mark>Appeared on: 2023-10-13</mark> -  _9 pages, 4 figures, accepted in ApJL_

</div>
<div id="authors">

A. R.~H.~Stevens, et al. -- incl., <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** $\noindent$ It has been shown in previous publications that the TNG100 simulation quantitatively reproduces the observed reduction in each of the total atomic and total molecular hydrogen gas for galaxies within massive halos, i.e. dense environments.In this Letter, we study how well TNG50 reproduces the $*resolved*$ effects of a Virgo-like cluster environment on the gas surface densities of satellite galaxies with $m_* > \! 10^9 {\rm M}_\odot$ and ${\rm SFR} \! > 0.05 {\rm M}_\odot{\rm yr}^{-1}$ .We select galaxies in the simulation that are analogous to those in the HERACLES and VERTICO surveys, and mock-observe them to the common specifications of the data.Although TNG50 does not quantitatively match the observed gas surface densities in the centers of galaxies, the simulation does qualitatively reproduce the trends of gas truncation and central density suppression seen in VERTICO in both $\HI$ and $\Htwo$ .This result promises that modern cosmological hydrodynamic simulations can be used to reliably model the post-infall histories of cluster satellite galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2310.08023/./Scaling_relations.png" alt="Fig2" width="100%"/>

**Figure 2. -** Resolved scaling relations for TNG50 galaxies per their field and cluster samples (and a high-mass field sub-sample), compared respectively with HERACLES and VERTICO.
Lines are running medians in 0.2-dex bins of $\Sigma_*$.
Shaded regions cover the 16th to 84th percentiles (not shown for the high-mass field sub-sample).
The left column accounts for all pixels in both the observations (provided they were detected in stellar emission) and simulations, setting non-detections in either \SHI or \SHtwo in the observations to zero, which are accounted for in the percentiles.
The right column removes any non-detections in the observations by cutting out any pixels that would fall below the axes as plotted.
The lower boundary of each axis represents the 1st percentile of all gas-detected pixels (irrespective of whether the pixel is a detection in \Ss) across both observational surveys.
This boundary also represents the cut in gas surface density applied to TNG50 in the right-hand panels, as to emulate a detection threshold.
The top two panels show the one-dimensional histograms of \Ss for pixels in each sample, normalised by the number of galaxies in that sample.
The $y$-axis in the top-right panel is stretched by a factor of two for clarity.
 (*fig:scalings*)

</div>
<div id="div_fig2">

<img src="tmp_2310.08023/./Individual_sequences_HI.png" alt="Fig3" width="100%"/>

**Figure 3. -** Individual \HI sequences for the nine most \HI-deficient galaxies among VERTICO analogues in TNG50.
Points are pixels from the TNG50 galaxies, with thick solid lines the running median of those points.
The thin, solid, red line that repeats in each panel is the median for the TNG50 field sample.
Each dashed line is the running median for the VERTICO galaxy that the TNG50 galaxy is matched to, based on its stellar mass and distance from the star-forming main sequence.
NGC4533 lacks any detected resolved \HI.
The lower bound of the $y$-axis in each panel is $\sim$0.5 dex lower than what is detected in VERTICO galaxies.
 (*fig:HI*)

</div>
<div id="div_fig3">

<img src="tmp_2310.08023/./HIH2frac_TNG100_TNG50.png" alt="Fig1" width="100%"/>

**Figure 1. -** The \HI (top panel) and \Htwo (bottom panel) fractions of TNG50 and TNG100 galaxies as a function of stellar mass at $z\!=\!0$.
Only galaxies with $m_* \! \geq \! 10^9 {\rm M}_\odot$ and ${\rm SFR} \! \geq 0.05 {\rm M_\odot yr}^{-1}$ are included (without any environmental sub-sampling).
Hex bins show the number density of TNG50 galaxies.
Lines are the running median (thick) and 16th and 84th percentiles (thin) for TNG50 (solid) and TNG100 (dashed).
Points with approximate errors compare the VERTICO and HERACLES galaxies that we use in this paper (a subset of the full surveys; cf. fig. 1 of  ([Zabel, Brown and Wilson 2022]()) ).
We show these observational data for reference, but we do *not* necessarily expect the simulation medians to align closely with them (but they should be closer to HERACLES than VERTICO).
Individual points from our TNG50 cluster and field samples, shown, can be respectively compared to VERTICO and HERACLES. (*fig:HIH2frac*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.08023"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\url}[1]{\href{#1}{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\}{aj}$
$\newcommand{\}{araa}$
$\newcommand{\}{apj}$
$\newcommand{\}{icarus}$
$\newcommand{\}{apjs}$
$\newcommand{\}{apjl}$
$\newcommand{\}{apss}$
$\newcommand{\}{aap}$
$\newcommand{\}{aapr}$
$\newcommand{\}{aaps}$
$\newcommand{\}{baas}$
$\newcommand{\}{memras}$
$\newcommand{\}{mnras}$
$\newcommand{\}{pasp}$
$\newcommand{\}{prl}$
$\newcommand{\}{jqsrt}$
$\newcommand{\}{actaa}$
$\newcommand{\pc}{{\rm pc}}$
$\newcommand{\lsun}{L_\odot}$
$\newcommand{\msun}{M_\odot}$
$\newcommand{\klms}{{\rm km s^{-1}}}$
$\newcommand{\kms}{{\rm km s^{-1}}}$
$\newcommand{ç}{{\rm cm^{-3}}}$
$\newcommand{\yr}{{\rm yr^{-1}}}$
$\newcommand{\sc}{{\rm cm^{-2}}}$
$\newcommand{\cm2}{{\rm cm^{-2}}}$
$\newcommand{\sqc}{{\rm cm^{-2}}}$
$\newcommand{\um}{{\rm \mu m}}$
$\newcommand{\micron}{{\rm \mu m}}$
$\newcommand{\htwo}{{\rm H_2}}$
$\newcommand{\hone}{{\rm HI}}$
$\newcommand{\nht}{{\rm NH_3}}$
$\newcommand{\hcn}{{\rm HCN}}$
$\newcommand{\nthp}{{\rm N_2H^+}}$
$\newcommand{\hctn}{{\rm HC_3N}}$
$\newcommand{\ceto}{{\rm C^{18}O}}$
$\newcommand{\hcop}{{\rm HCO^+}}$
$\newcommand{\chtcn}{{\rm CH_3CN}}$
$\newcommand{\htcop}{{\rm H^{13}CO^+}}$
$\newcommand{\chtoh}{{\rm CH_3OH}}$
$\newcommand{\htcn}{{\rm H^{13}CN}}$
$\newcommand{\tco}{{\rm ^{13}CO}}$
$\newcommand{çh}{{\rm CCH}}$
$\newcommand{\cth}{{\rm C_2H}}$
$\newcommand{\mjbm}{{\rm mJy  beam^{-1}}}$
$\newcommand{\}{natexlab}$</div>



<div id="title">

# A High-Mass Young Star-forming Core Escaping from Its Parental Filament

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.08180-b31b1b.svg)](https://arxiv.org/abs/2310.08180)<mark>Appeared on: 2023-10-13</mark> -  _15 pages, 6 figures, accepted for publication in the Astrophysical Journal_

</div>
<div id="authors">

Z. Ren, et al. -- incl., <mark>S. Li</mark>

</div>
<div id="abstract">

**Abstract:** We studied the unique kinematic properties in massive filament G352.63-1.07 at $10^3$ -AU spatial scale with the dense molecular tracers observed with the Atacama Large Millimeter/submillimeter Array (ALMA). We find the central massive core M1 ( $12 \msun$ ) being separated from the surrounding filament with a velocity difference of $v- \overline{v}_{\rm sys}=-2 \kms$ and a transverse separation within 3 arcsec. Meanwhile, as shown in multiple dense-gas tracers, M1 has a spatial extension closely aligned with the main filament and is connected to the filament towards its both ends. M1 thus represents a very beginning state for a massive young star-forming core escaping from the parental filament, within a time scale of $\sim 4000$ years. Based on its kinetic energy ( $3.5\times10^{44}$ erg), the core escape is unlikely solely due to the original filament motion or magnetic field, but requires more energetic events such as a rapid intense anisotropic collapse. The released energy also seems to noticeably increase the environmental turbulence. This may help the filament to become stabilized again.

</div>

<div id="div_fig1">

<img src="tmp_2310.08180/./m1_blue3.png" alt="Fig2" width="100%"/>

**Figure 2. -** \small** (a)** Contours: emission region of the $\htcop$(1-0) line in three velocity intervals. The background image is the $\htcop$ emission in (-1.5,+1.5) $\kms$(red-center component). For the blue-wing component, the contour levels are 4, 6, and 8 times of the rms level (0.3 K $\kms$). For the other components, the contour levels are 10\% to 90\% in 20\%-step of the peak intensity, which is 15, 11, and 3 K $\kms$ for the blue-center, red-center, and red-wing, respectively. The green arrows label the mass transfer flow directions onto the main filament (Chen21). The green circles labels the possible arrival points of the transfer flows onto the filament. ** (b)** PV plot and intensity profile along the major axis of the main filament. The sampling direction is labelled in dashed line in panel (a). The vertical dashed lines denote the projected offset of the three dense cores on the sampling direction. The horizontal dotted line represents the average systemic velocity of the filament.  (*fig:m1_blue*)

</div>
<div id="div_fig2">

<img src="tmp_2310.08180/./spec_img.png" alt="Fig1" width="100%"/>

**Figure 1. -** \small** (a)** The velocity-integrated intensity of the $\htcop$(1-0) line (false-color) and 3 mm continuum. The contour levels are $4\sigma_{\rm rms}$($1.6 $\mjbm$$) to $84\sigma_{\rm rms}$(peak) in step of $16\sigma_{\rm rms}$.  ** (b)** The $\htcop$ and $\chtoh$ emission regions around the main filament, overlaid on the IRAC-RGB image (3.6, 4.5, and 8.0 $\micron$ bands). The $\htcop$ contours are 15\% to 90\% in 15\%-step of the peak intensity (8.5 K $\kms$). The $\chtoh$ contours are 10\% to 90\% in 20\%-step of the peak intensity (18.7 K $\kms$). The dashed circles labels the area of each core. ** (c)** The spectra at the selected core centers. The blue and red-shaded areas indicate the velocity ranges of the two velocity components, respectively. The vertical dashed line denotes the division between the blue and main-filament components.  (*fig:spec_img*)

</div>
<div id="div_fig3">

<img src="tmp_2310.08180/./pv_3mol.png" alt="Fig3" width="100%"/>

**Figure 3. -** \small** Left column:** The emission regions of the blue- (contours) and redshifted (false-color) components in CCS and $\htcn$(1-0) lines. The contour levels are 20\% to 90\% of the peak intensity. ** Right column:** The PV diagrams of the two molecular lines. the $\htcn$ emission at $v_{\rm lsr}=2$ to 4 $\kms$ is from anther HFC of $F=1-1$. The HFCs of $\htcn$ are separated for 6-7 $\kms$ and would not blend with each other. The CCH (1-0) emission shows an additional small blueshift wings around M3, which should correspond to the transfer flow onto the filament (Chen21). This feature is not seen in other lines probably because of their lower optical depths.  (*fig:pv_3mol*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.08180"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

300  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

12  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
